In [6]:
from bs4 import BeautifulSoup
import requests

In [14]:
BASE_URL = "https://www.metacritic.com/browse/games/score/metascore/all/all?page=0"

# Need a header, so that the Website didn't think, we are a bot
headers = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36"}

In [21]:
# Use request library to call the webpage with headers as headers
page = requests.get("https://www.metacritic.com/browse/games/score/metascore/all/all?page=0", headers = headers)

In [18]:
# Create instance of BeautifulSoup with the html content of the page for scraping
soup = BeautifulSoup(page.content, "html.parser")

In [32]:
games = soup.find_all('td', class_='clamp-summary-wrap')

In [55]:
#Extracting the rank
int(games[0].find('span', class_='title numbered').text.strip().strip('.'))

1

In [50]:
#Extracting the title of the game
games[0].find('a', class_='title').text

'The Legend of Zelda: Ocarina of Time'

In [49]:
#Extracting platform/console game has been released on
games[0].find('div', class_='platform').find('span', class_='data').text.strip()

'Nintendo 64'

In [61]:
#Release date
games[0].find('div', class_='clamp-details').find_all('span')[2].text

'November 23, 1998'